# Interrogation of Rosetta Stone Data Using Python’s Altair for Dynamically Linked Data

The following is an Altair plot of Clerke’s Rosetta Stone data used for calibration of our Thomeer capillary pressure parameters and Petrophysical Rock Types (PRT). We start by selecting a small sampling of samples from the porosity vs. permeability cross plot, and then Altair shows only the Capillary Pressure curves for the selected samples, a whisker plot of the mode of the mode of the pore throat radius as well as the Pore Throat Distribution plots for all samples or just the selected samples. 


In [1]:
#!/usr/bin/env python3
# python loglan
# When referencing Geolog variables in a Python script, names must always be lower case.

# Imports

import numpy as np

# python loglan
# When referencing Geolog variables in a Python script, names must always be lower case.

# Imports

from pandas import DataFrame, read_csv

import altair as alt
#alt.renderers.enable('altair_viewer')

import pandas as pd

import altair_transform

alt.data_transformers.disable_max_rows()


#read the file
file = r'./data/Pc_Results.xlsx'
df = pd.read_excel(file,index_col=False)


interval = alt.selection_interval()

base = alt.Chart(df).properties(
    width=300,
    height=375, 
).add_selection(interval)



kphi = alt.Chart(df).mark_point(filled=True, size=100).encode(
    x='Porosity:Q',
    y='lPerm:Q',
    #size='lMode:Q',
    color=alt.condition(interval, 'ROCK_INDEX:O', alt.value('lightgray'),scale=alt.Scale(scheme='rainbow')),
    tooltip='Mode:Q', 
).properties(
    width=500,
    height=500, 

    title='Porosity vs. Permeability Cross Plot (PRT on Color Axis)',
    selection=interval
)

pc = alt.Chart(df).mark_line(clip=True).encode(
    alt.X('BVOCC:Q',
        scale=alt.Scale(domain=(30, 0))
    ),    
    y='lPc:Q',
    color=alt.condition(interval, 'ROCK_INDEX:O', alt.value('lightgray'),scale=alt.Scale(scheme='rainbow')),
    tooltip='Mode:Q', 
    strokeDash='Sample:N',
).properties(
    title='Pc Curves with PRT on Color Axis (Upscaled Pc curve is thick Black Curve)',
    width=500,
    height=500, 
    
).transform_filter(
    interval
)

pc2 = alt.Chart(df).mark_line(strokeWidth=6).encode(
    alt.X('BVOCC:Q', aggregate='mean',
          scale=alt.Scale(domain=(30, 0))
         ),
    alt.Y('lPc:Q'),
    color=alt.value('black'),
).properties(
    #title='Upscaled Pc Curve with PRT as Color Axis',
    width=500,
    height=500, 
    
).transform_filter(
    interval
)
    
ptd = base.mark_line( clip=True).encode(
    alt.X('lMicrons:Q',
        scale=alt.Scale(domain=(-2., 2.5))
    ),    
    alt.Y('dBv:Q',
        scale=alt.Scale(domain=(0, 15))
    ),    
    color=alt.condition(interval, 'ROCK_INDEX:O', alt.value('lightgray'),scale=alt.Scale(scheme='rainbow')),
    strokeDash='Sample:N',
).properties(
    title='Pore Throat Distribution (Microns)',
    width=400,
    height=200,
)

ptd2 = base.mark_line( clip=True).encode(
    alt.X('lMicrons:Q',
        scale=alt.Scale(domain=(-2., 2.5))
    ),    
    alt.Y('dBv:Q',
        scale=alt.Scale(domain=(0, 15))
    ),    
    color=alt.condition(interval, 'ROCK_INDEX:O', alt.value('lightgray'),scale=alt.Scale(scheme='rainbow')),
    strokeDash='Sample:N',
).properties(
    title='Pore Throat Distribution (Microns)',
    width=400,
    height=200,
).transform_filter(
    interval
)


hist2 = alt.Chart(df).mark_bar().encode(
    alt.X('ROCK_INDEX:O'),
    #x='rock_index:O',
    #y='lmode:Q',
    alt.Y('Mode:Q', scale=alt.Scale(type='log', base=10, domain=(0.001, 100.0)), axis=alt.Axis(orient='left') ),
    color=alt.condition(interval, 'ROCK_INDEX:O', alt.value('lightgray'),scale=alt.Scale(scheme='rainbow'))
).properties(
    title='Mode of Pore Throats by Petrophysical Rock Type (PRT)',
    width=400,
    height=500,
    #selection=interval
)


box2 = alt.Chart(df).mark_boxplot(extent='min-max').encode(
    alt.X('ROCK_INDEX:O'),
    alt.Y('Mode:Q', scale=alt.Scale(type='log', base=10, domain=(0.001, 100.0)), axis=alt.Axis(orient='left') ),
    #color=alt.condition(interval, 'rock_index:O', alt.value('lightgray'),scale=alt.Scale(scheme='rainbow')),
    color='ROCK_INDEX:O',      
).properties(
    title='Box Plot of Mode of PTD by PRT ',
    width=400,
    height=500, 

).transform_filter(
    interval
)




#kphihist = kphi | hist2
kphihist = kphi | box2

pchist = pc + pc2 | ptd & ptd2


plot = kphihist & pchist 



plot







alt.VConcatChart(...)